In [37]:
import os
from datetime import datetime, timedelta
from os.path import dirname, basename

import git
import pandas as pd
import plotly.express as px

# here it's good to get our bearings

In [38]:
os.getcwd()

# read a repository to graph

In [39]:
wfx = "/Users/timottinger/Projects/siemens-wfx"
goreleaser = "/Users/timottinger/Projects/goreleaser"
repo = git.Repo(goreleaser)

In [40]:
repo_title = basename(dirname(repo.common_dir))

In [41]:
def create_tag_interval_df(repo: git.Repo, since: datetime) -> pd.DataFrame:
    source = ((tag_ref.name, tag_ref.commit.authored_datetime) for tag_ref in repo.tags)
    raw_df = pd.DataFrame(data=source, columns=["name", "timestamp"])
    if raw_df.empty:
        return raw_df
    raw_df["timestamp"] = pd.to_datetime(raw_df["timestamp"], utc=True)
    sorted_df = raw_df.sort_values(by=["timestamp"])
    graph_df = sorted_df[sorted_df['timestamp'] > since].copy()
    graph_df['interval'] = graph_df['timestamp'].diff()
    return graph_df


print("Okay")

we will need to generate a dataframe for the tags, with label and timestamp

Let's filter this down to the given date and later

In [42]:

start_date = datetime.today().astimezone() - timedelta(days=60)
graph_df = create_tag_interval_df(repo, start_date)
graph_df

This should give us something to work with

In [43]:
figure = px.bar(graph_df, x="timestamp", y="interval", title='Release Delay')
figure.show()


In [44]:
earlier_label = graph_df.iloc[0]['name']
diffcounts = []
for index, data in graph_df.iterrows():
    current_label = data['name']
    diffs = repo.commit(earlier_label).diff(current_label)
    diffcounts.append(len(diffs))
    earlier_label = current_label

diffcounts


In [45]:
graph_df['timestamp'] = pd.to_datetime(graph_df['timestamp'], utc=True)

In [46]:
graph_df['diff_counts'] = diffcounts

In [47]:
graph_df

In [48]:
figure = px.scatter(
    graph_df.iloc[-25:],
    x="timestamp",
    y="diff_counts",
    size="diff_counts",
    color="diff_counts",
    title=f'{repo_title} Release Size and Frequency',
    hover_data=["timestamp", "name", "interval"]
)
figure.update_layout(autosize=True)
figure.update_traces(mode="markers+lines")
figure.show()

In [49]:
dir(figure)

In [50]:
hovertemplate = "%{name}"
figure = px.line(
    graph_df,
    title=f'{repo_title} Release Frequency and Size',
    x="timestamp",
    y="diff_counts",
    hover_name="name",
    markers=True
)
#figure.update_traces(mode="markers")
figure.update_layout(hovermode="x unified")
figure.show()

In [51]:
graph_df.info()


Add a change-type stacked graph based on diffs.


In [52]:
by_date = lambda x: x.commit.authored_datetime
sorted_tags = sorted(repo.tags, key=by_date)

In [53]:

last_20 = sorted_tags[-20:]
for x in last_20:
    print(by_date(x), x)


In [54]:

start = last_20[0]
start


In [55]:
from collections import Counter

change_name = {
    "A": "Files Added",
    "D": "Files Deleted",
    "R": "Files Renamed",
    "M": "Files Modified"
}


def change_series(start, last_20):
    for tag in last_20:
        diffs = start.commit.diff(tag.commit)
        yield {
            'Date': start.commit.committed_datetime.date(),
            'Name': start.name,
            **Counter(change_name[x.change_type] for x in diffs)
        }
        start = tag


change_df = pd.DataFrame(change_series(start, last_20))
change_df

In [56]:
figure = px.bar(
    change_df,
    title=f"Changes Across Tags for {repo_title}",
    x="Name",
    y=list(change_name.values()),
    labels=["Added", "Deleted", "Modified", "Removed"],
    hover_name="Name",
    hover_data=["Date"],
    text_auto='.2s'
)
figure.show()

In [57]:
figure = px.area(
    change_df,
    title=f"Changes Across Tags for {repo_title}",
    x="Name",
    y=list(change_name.values()),
    labels=["Added", "Deleted", "Modified", "Removed"],
    hover_name="Name",
    hover_data=["Date"],
)
figure.show()